In [2]:
# Section 1: Training a Simple Model

import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from torch.utils.data import DataLoader, TensorDataset

# Load and prepare the dataset
iris = load_iris()
X = iris.data
y = iris.target.reshape(-1, 1)

# One-hot encode the target variable
encoder = OneHotEncoder(sparse=False)
y = encoder.fit_transform(y)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

# Create DataLoader
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=5, shuffle=True)

# Define the model
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(X_train.shape[1], 10)
        self.fc2 = nn.Linear(10, 10)
        self.fc3 = nn.Linear(10, y_train.shape[1])

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.softmax(self.fc3(x), dim=1)
        return x

model = SimpleNN()

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
for epoch in range(50):
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets.argmax(dim=1))
        loss.backward()
        optimizer.step()
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/50], Loss: {loss.item():.4f}')

# Evaluate the model
with torch.no_grad():
    outputs = model(X_test)
    _, predicted = torch.max(outputs, 1)
    _, labels = torch.max(y_test, 1)
    accuracy = (predicted == labels).float().mean()
    print(f'Test Accuracy: {accuracy:.4f}')

# Save the model in PyTorch format
torch_model_path = './output/iris_model_pytorch.pth'
torch.save(model.state_dict(), torch_model_path)
print(f'PyTorch model saved to {torch_model_path}')


c:\Users\Amr osama abdellatif\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Epoch [10/50], Loss: 0.9038
Epoch [20/50], Loss: 0.7324
Epoch [30/50], Loss: 0.7290
Epoch [40/50], Loss: 0.6058
Epoch [50/50], Loss: 0.5859
Test Accuracy: 0.9333
PyTorch model saved to ./output/iris_model_pytorch.pth


In [3]:
# Section 2: Conversion to ONNX

import torch.onnx

# Load the saved PyTorch model
model = SimpleNN()
model.load_state_dict(torch.load(torch_model_path))
model.eval()

# Define the model path
onnx_model_path = './output/iris_model.onnx'

# Convert the PyTorch model to ONNX
dummy_input = torch.tensor(X_train[0:1], dtype=torch.float32)
torch.onnx.export(model, dummy_input, onnx_model_path, 
                  input_names=['input'], output_names=['output'], 
                  dynamic_axes={'input': {0: 'batch_size'}, 'output': {0: 'batch_size'}},
                  opset_version=13)

print(f'ONNX model saved to {onnx_model_path}')


ONNX model saved to ./output/iris_model.onnx


C:\Users\Amr osama abdellatif\AppData\Local\Temp\ipykernel_23196\2470347034.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  dummy_input = torch.tensor(X_train[0:1], dtype=torch.float32)


In [4]:
# Section 3: Inference

import onnxruntime as ort
import numpy as np

# Load the ONNX model
onnx_model = ort.InferenceSession(onnx_model_path)

# Provided data for inference
data = np.array([[4.5, 4.9, 5.1, 5.4],
                 [1.5, 2.9, 3.1, 1.4],
                 [7.5, 6.9, 8.1, 6.4]], dtype=np.float32)

def prepare_input(data):
    return {onnx_model.get_inputs()[0].name: data}

# Make predictions
input_data = prepare_input(data)
predictions = onnx_model.run(None, input_data)

# Convert the predictions to class labels
predicted_labels = np.argmax(predictions[0], axis=1)
print(f'Predicted labels: {predicted_labels}')


Predicted labels: [2 2 2]
